In [1]:
import pandas as pd
import numpy as np

from bs4 import BeautifulSoup
from scrapy.selector import Selector

import re, requests, csv, html
import lxml

# Regex
regex_html = r"(<[^>]+>)"
regex_html_4_content = r"(\<\/?[a-zA-Z0-9\=\"\_\s\-]+\>)"
regex_d_space = r"(\s{2,})"
regex_weblink = r"(www.[A-Za-z\.]+)"

# File url
file_url = './training/jobs.csv'
dest_url = './training/training.csv'


# Basic list
csv_header = ['title','location','type_empl','wage','industries','content']

# Support scripts
# %run -i scripts.py

C:\Users\Shaynie\Anaconda3\envs\capstone_app\lib\site-packages\requests\__init__.py:91: RequestsDependencyWarning: urllib3 (1.25.2) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [2]:
# Merge list of string into a long text.
def join_txt(list_txt):
    # Join list of content into 1 simple paragraph
    seperator = '. '
    return seperator.join(list_txt)

# Clear any html tags and decode the text
def clear_html_and_unescape(sample):
    # Clear html
    result = re.sub(regex_html, '', sample[0])
    
    # Clear Unescape
    result = html.unescape(result)
    
    return result

# Clean Web content
def clean_web_scrap_content(content):
    
    content = join_txt(content)
    content = re.sub(r"(<[^>]+>)",' ',content)
    content = html.unescape(content)
    content = re.sub(r"(\s{2,})", ' ',content)
    content = re.sub(r"(<[^>]+>)",'website',content)
    content = content.replace(u'\xa0', u' ').lstrip().rstrip()
    content = re.sub(r'([\•\s]{2,})',' ',content)
    
    return content

In [3]:
frame = pd.read_csv(file_url, sep=',')
frame.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8354 entries, 0 to 8353
Data columns (total 1 columns):
url    8354 non-null object
dtypes: object(1)
memory usage: 65.3+ KB


In [4]:
# -------------------------------------------------
# Grant Url as a list
frame_urls = frame['url'].values

# Attempt title
title_attempt = True
for key, url in enumerate(frame_urls):
    # XPath Strings
    res = requests.get(url)
    if res.status_code  == 200:

        print("Header: ", res.status_code)
        # Initialize XPath
        xpath_selector = Selector(text=res.text)
        
        # Init dictionary
        list_content = dict()
        
        # Print url for tracking
        print(url)

        # Title
        # //h1[@class="jobtitle"]
#         title = xpath_selector.xpath('//h1[@class="jobtitle"]/text()').extract()
        title = xpath_selector.xpath('//title/text()').extract()
        title = title[0]

        # Location
        # //section[@role="region"]/dl/dd[2]/span/span
        location = xpath_selector.xpath('//section[@role="region"]/dl/dd[2]/span/span').extract()
        location = clear_html_and_unescape(location)

        # Type of Employee
        # //section[@role="region"]/dl/dd[3]/span/span
        type_employee = xpath_selector.xpath('//section[@role="region"]/dl/dd[3]/span/span').extract()
        type_employee = re.sub(regex_html,'', type_employee[0])

        # Wage
        # //*[contains(text(),'Salary')]
        is_Salary = xpath_selector.xpath('//*[contains(text(),"Salary")]/text()').extract()
        if is_Salary:
            # TODO
            wage = xpath_selector.xpath('//article/div[1]/div[2]/div/section/dl/div[1]/dd/span/span/text()').extract()
            if wage:
                wage = wage[0]
            else:
                wage = is_Salary
        else:
            wage = ''

        # Industries
        # //section[@role="region"]/dl/div/dd/span/span/strong
        industries = xpath_selector.xpath('//section[@role="region"]/dl/div/dd/span/span/strong').extract()
        industries = clear_html_and_unescape(industries)

        # Content
        # //div[@data-automation="mobileTemplate"]
        content = xpath_selector.xpath('//div[@data-automation="mobileTemplate"]').extract()
        content = clean_web_scrap_content(content)
        
        # Append to dictionary
        list_content.update({
            'title':     title,
            'location':  location,
            'type_empl': type_employee,
            'wage':      wage,
            'industries':industries,
            'content':   content
        })
        
        if title_attempt:
            print("TItle: ", title_attempt)
            df_temp = pd.DataFrame(columns = list(list_content.keys()))
            frame = pd.concat([frame, df_temp])
            print(frame.columns)                
            title_attempt = False

        for k_, v_ in list_content.items():
            print(k_," : " ,len(v_))
            frame.loc[key,k_] = v_
        print("======================\n")
    else:
        print("['ERRORS'] {} , at {}".format(res.status_code, url))

['ERRORS'] 410 , at https://www.seek.com.au/job/38884116?type=promoted&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
Header:  200
https://www.seek.com.au/job/38983503?type=standard&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
TItle:  True
Index(['content', 'industries', 'location', 'title', 'type_empl', 'url',
       'wage'],
      dtype='object')
title  :  65
location  :  41
type_empl  :  9
wage  :  0
industries  :  20
content  :  1867



C:\Users\Shaynie\Anaconda3\envs\capstone_app\lib\site-packages\ipykernel_launcher.py:74: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



Header:  200
https://www.seek.com.au/job/38980384?type=standout&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
title  :  84
location  :  38
type_empl  :  13
wage  :  0
industries  :  38
content  :  2902

Header:  200
https://www.seek.com.au/job/39005312?type=standard&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
title  :  48
location  :  41
type_empl  :  9
wage  :  0
industries  :  25
content  :  2233

Header:  200
https://www.seek.com.au/job/38991024?type=standout&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
title  :  92
location  :  41
type_empl  :  9
wage  :  0
industries  :  10
content  :  5096

Header:  200
https://www.seek.com.au/job/38995358?type=standard&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
title  :  45
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  2843

Header:  200
https://www.seek.com.au/job/38988764?type=standout&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733
title  :  60
location  :  41
ty

Header:  200
https://www.seek.com.au/job/38885762?type=standard&searchrequesttoken=94ad9abc-5d85-4b1e-9bed-75c0141c316e
title  :  59
location  :  41
type_empl  :  13
wage  :  0
industries  :  38
content  :  1739

Header:  200
https://www.seek.com.au/job/38974336?type=standard&searchrequesttoken=94ad9abc-5d85-4b1e-9bed-75c0141c316e
title  :  54
location  :  41
type_empl  :  9
wage  :  0
industries  :  20
content  :  2206

Header:  200
https://www.seek.com.au/job/38911655?type=standard&searchrequesttoken=94ad9abc-5d85-4b1e-9bed-75c0141c316e
title  :  100
location  :  41
type_empl  :  9
wage  :  50
industries  :  38
content  :  1699

Header:  200
https://www.seek.com.au/job/38948829?type=standout&searchrequesttoken=94ad9abc-5d85-4b1e-9bed-75c0141c316e
title  :  37
location  :  29
type_empl  :  9
wage  :  0
industries  :  38
content  :  2609

Header:  200
https://www.seek.com.au/job/38925158?type=standout&searchrequesttoken=bc23cd76-ea59-4d48-aaec-75058e2ceb93
title  :  60
location  :  29


Header:  200
https://www.seek.com.au/job/38919897?type=standard&searchrequesttoken=2ff4a75d-d9f5-4e5a-b973-daed292d5660
title  :  45
location  :  41
type_empl  :  9
wage  :  13
industries  :  28
content  :  1483

['ERRORS'] 410 , at https://www.seek.com.au/job/38853207?type=standout&searchrequesttoken=2ff4a75d-d9f5-4e5a-b973-daed292d5660
Header:  200
https://www.seek.com.au/job/38974353?type=standard&searchrequesttoken=2ff4a75d-d9f5-4e5a-b973-daed292d5660
title  :  58
location  :  41
type_empl  :  9
wage  :  0
industries  :  20
content  :  1958

Header:  200
https://www.seek.com.au/job/38996476?type=standout&searchrequesttoken=2ff4a75d-d9f5-4e5a-b973-daed292d5660
title  :  46
location  :  41
type_empl  :  9
wage  :  0
industries  :  28
content  :  2328

Header:  200
https://www.seek.com.au/job/38885939?type=standout&searchrequesttoken=2ff4a75d-d9f5-4e5a-b973-daed292d5660
title  :  66
location  :  6
type_empl  :  9
wage  :  0
industries  :  21
content  :  2829

Header:  200
https://www.

...

Header:  200
https://www.seek.com.au/job/38869385?type=standout&searchrequesttoken=307c8d62-cd0b-4c06-9d1b-c84be2915f9c
title  :  41
location  :  41
type_empl  :  9
wage  :  27
industries  :  38
content  :  1067

Header:  200
https://www.seek.com.au/job/38886002?type=standout&searchrequesttoken=107aa309-ebe1-488c-8c6f-879d724ab00b
title  :  44
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  4855

Header:  200
https://www.seek.com.au/job/38860650?type=standout&searchrequesttoken=107aa309-ebe1-488c-8c6f-879d724ab00b
title  :  57
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  3307

Header:  200
https://www.seek.com.au/job/38883576?type=standard&searchrequesttoken=107aa309-ebe1-488c-8c6f-879d724ab00b
title  :  46
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  2901

Header:  200
https://www.seek.com.au/job/38864597?type=standard&searchrequesttoken=107aa309-ebe1-488c-8c6f-879d724ab00b
title  :  55
location  :  41
ty

Header:  200
https://www.seek.com.au/job/38886421?type=standard&searchrequesttoken=e3815c9b-a0fe-4cb0-821b-7eac029d51fa
title  :  80
location  :  29
type_empl  :  9
wage  :  3
industries  :  20
content  :  6804

Header:  200
https://www.seek.com.au/job/38996206?type=standout&searchrequesttoken=e3815c9b-a0fe-4cb0-821b-7eac029d51fa
title  :  43
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  2035

Header:  200
https://www.seek.com.au/job/36501930?type=standard&searchrequesttoken=88cbe663-46e6-4baf-94f8-6fbe669d9f9b
title  :  38
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  1309

Header:  200
https://www.seek.com.au/job/38947920?type=standard&searchrequesttoken=88cbe663-46e6-4baf-94f8-6fbe669d9f9b
title  :  54
location  :  36
type_empl  :  13
wage  :  16
industries  :  38
content  :  2705

Header:  200
https://www.seek.com.au/job/38825541?type=standout&searchrequesttoken=88cbe663-46e6-4baf-94f8-6fbe669d9f9b
title  :  58
location  :  41
t

Header:  200
https://www.seek.com.au/job/38959219?type=standard&searchrequesttoken=85d1e5fc-3500-4a6e-aa95-a5ab3c46ae69
title  :  62
location  :  41
type_empl  :  9
wage  :  16
industries  :  38
content  :  2585

Header:  200
https://www.seek.com.au/job/38980827?type=standout&searchrequesttoken=85d1e5fc-3500-4a6e-aa95-a5ab3c46ae69
title  :  85
location  :  41
type_empl  :  9
wage  :  17
industries  :  20
content  :  7314

Header:  200
https://www.seek.com.au/job/38985629?type=standard&searchrequesttoken=b0fcbf0a-70ff-45aa-98c6-e74a2db5912b
title  :  64
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  2326

Header:  200
https://www.seek.com.au/job/38919726?type=standard&searchrequesttoken=b0fcbf0a-70ff-45aa-98c6-e74a2db5912b
title  :  48
location  :  41
type_empl  :  9
wage  :  26
industries  :  38
content  :  1737

Header:  200
https://www.seek.com.au/job/38916842?type=standard&searchrequesttoken=b0fcbf0a-70ff-45aa-98c6-e74a2db5912b
title  :  98
location  :  41


['ERRORS'] 410 , at https://www.seek.com.au/job/38914798?type=standard&searchrequesttoken=183a0410-d6ba-4c45-9c9f-11fc5c9f50b5
Header:  200
https://www.seek.com.au/job/38908774?type=standout&searchrequesttoken=183a0410-d6ba-4c45-9c9f-11fc5c9f50b5
title  :  71
location  :  38
type_empl  :  9
wage  :  0
industries  :  21
content  :  2806

['ERRORS'] 410 , at https://www.seek.com.au/job/38950999?type=standout&searchrequesttoken=183a0410-d6ba-4c45-9c9f-11fc5c9f50b5
['ERRORS'] 410 , at https://www.seek.com.au/job/38950998?type=standout&searchrequesttoken=a5643cdb-7f00-4e1b-aebc-1e321a2d3dd0
Header:  200
https://www.seek.com.au/job/38938460?type=standout&searchrequesttoken=a5643cdb-7f00-4e1b-aebc-1e321a2d3dd0
title  :  45
location  :  41
type_empl  :  9
wage  :  0
industries  :  38
content  :  3219

Header:  200
https://www.seek.com.au/job/39006456?type=standout&searchrequesttoken=a5643cdb-7f00-4e1b-aebc-1e321a2d3dd0
title  :  40
location  :  36
type_empl  :  9
wage  :  0
industries  :  36
c

Header:  200
https://www.seek.com.au/job/38848979?type=standard&searchrequesttoken=53883ad2-8054-4463-9753-52778278a894
title  :  37
location  :  29
type_empl  :  9
wage  :  0
industries  :  38
content  :  1936

Header:  200
https://www.seek.com.au/job/38882157?type=standard&searchrequesttoken=53883ad2-8054-4463-9753-52778278a894
title  :  39
location  :  41
type_empl  :  9
wage  :  0
industries  :  26
content  :  4220

Header:  200
https://www.seek.com.au/job/38935694?type=standout&searchrequesttoken=53883ad2-8054-4463-9753-52778278a894
title  :  31
location  :  35
type_empl  :  9
wage  :  0
industries  :  36
content  :  3068

Header:  200
https://www.seek.com.au/job/38914801?type=standard&searchrequesttoken=53883ad2-8054-4463-9753-52778278a894
title  :  50
location  :  41
type_empl  :  9
wage  :  42
industries  :  5
content  :  3358

Header:  200
https://www.seek.com.au/job/38849400?type=standout&searchrequesttoken=53883ad2-8054-4463-9753-52778278a894
title  :  37
location  :  29
typ

In [5]:
frame.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8354 entries, 0 to 8353
Data columns (total 7 columns):
content       7769 non-null object
industries    7769 non-null object
location      7769 non-null object
title         7769 non-null object
type_empl     7769 non-null object
url           8354 non-null object
wage          7769 non-null object
dtypes: object(7)
memory usage: 842.1+ KB


In [6]:
frame.head()

,content,industries,location,title,type_empl,url,wage
0,NaN,NaN,NaN,NaN,NaN,https://www.seek.com.au/job/38884116?type=prom...,NaN
1,"Our client, a respected and leading global ban...",Science & Technology,"Sydney, CBD, Inner West & Eastern Suburbs",Senior Analyst | Customer Insights | Banking J...,Full Time,https://www.seek.com.au/job/38983503?type=stan...,
2,We are looking for an experienced Senior Busin...,Information & Communication Technology,"Sydney, North Shore & Northern Beaches","Senior Business Analyst, Data & Analytics (Cor...",Contract/Temp,https://www.seek.com.au/job/38980384?type=stan...,
3,Senior Data Analysts - check this out! This is...,"Advertising, Arts & Media","Sydney, CBD, Inner West & Eastern Suburbs",Data and Analytics Director Job in Sydney - SEEK,Full Time,https://www.seek.com.au/job/39005312?type=stan...,
4,We'll have you working with Australia's most r...,Accounting,"Sydney, CBD, Inner West & Eastern Suburbs",Forensic Technology/Data Analytics - Senior Co...,Full Time,https://www.seek.com.au/job/38991024?type=stan...,


In [7]:
frame['content']=frame['content'].map(lambda row: re.sub(r'([\•\s]{2,})',' ',str(row)))

In [8]:
frame['content'][0]

'nan'

In [ ]:
frame.to_csv(dest_url,sep='|',index=False)

In [ ]:
# Test Cell

url_w_salary = 'https://www.seek.com.au/job/38961481?type=standout&searchrequesttoken=500a5225-80f9-4fcf-88c4-4985e717adf2'
url_w_salary_1 = 'https://www.seek.com.au/job/38988764?type=standout&searchrequesttoken=7dd0818a-ae26-4408-918c-981552979733'
url_wo_salary = 'https://www.seek.com.au/job/38952364?type=standout&searchrequesttoken=bd45b375-6e0c-4572-852c-1bfa7acc2952'

url_generous_salary = 'https://www.seek.com.au/job/38931990?type=standard&searchrequesttoken=ddc9ad23-06e8-42c5-9691-a35a9f7c6c5e'
url_test_random = 'https://www.seek.com.au/job/38843200?type=standard&searchrequesttoken=480f435f-3db5-4612-a554-a31df99c685e'

# XPath Strings
res = requests.get(url_generous_salary)
if res.status_code  == 200:

    print("Header: ", res.status_code)
    # Initialize XPath
    xpath_selector = Selector(text=res.text)
    
    # Title
    # //h1[@class="jobtitle"]
    title = xpath_selector.xpath('//h1[@class="jobtitle"]/text()').extract()

    # Location
    # //section[@role="region"]/dl/dd[2]/span/span
    location = xpath_selector.xpath('//section[@role="region"]/dl/dd[2]/span/span').extract()
    location = clear_html_and_unescape(location)
    
    # Type of Employee
    # //section[@role="region"]/dl/dd[3]/span/span
    type_employee = xpath_selector.xpath('//section[@role="region"]/dl/dd[3]/span/span').extract()
    type_employee = re.sub(regex_html,'', type_employee[0])
    
    # Wage
    # //*[contains(text(),'Salary')]
    is_Salary = xpath_selector.xpath('//*[contains(text(),"Salary")]/text()').extract()
    if is_Salary:
        # TODO
        wage = xpath_selector.xpath('//article/div[1]/div[2]/div/section/dl/div[1]/dd/span/span/text()').extract()
        wage = wage[0]
    else:
        wage = ''
        
    # Industries
    # //section[@role="region"]/dl/div/dd/span/span/strong
    industries = xpath_selector.xpath('//section[@role="region"]/dl/div/dd/span/span/strong').extract()
    industries = clear_html_and_unescape(industries)
    
    # Content
    # //div[@data-automation="mobileTemplate"]
    content = xpath_selector.xpath('//div[@data-automation="mobileTemplate"]').extract()
    content = clean_web_scrap_content(content)
    
    # Print-out
    print("Title: ", title[0])
    print("Location: ", location)
    print("Type Empl: ", type_employee)
    print("Wage: ", wage)
    print("Industries: ", industries)
    print("Content: ", len(content))
else:
    print("[ERROR]: ", res.status_code)

In [ ]:
content